In [1]:
import time
import torch.optim
import torch.nn as nn
from torchvision import models
from torch.autograd import Variable
from datasets.dataset import CUBDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time

In [2]:
learning_rate = 1e-3
input_channels = 1
output_features = 6
epoch = 1
save_model_name = 'models/resnet.pth'

In [3]:
trainset = CUBDataset()
trainloader = DataLoader(dataset=trainset, batch_size=10, shuffle=True)
testset = CUBDataset(is_test = True)
testloader = DataLoader(dataset=testset, batch_size=10, shuffle=True)

In [5]:
#conv = torch.hub.load('pytorch/vision:v0.9.0', 'vgg19', pretrained=True)
#conv.eval()
conv = models.resnet18()
conv.conv1 = nn.Conv2d(input_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
conv.fc = nn.Linear(in_features=512, out_features=output_features, bias=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(conv.parameters(), lr=learning_rate)

In [5]:
epoch_loss = list()

for epoch_number in range(epoch):
    running_loss, count, acc = 0., 0, 0.
    for img, label in trainloader:
        t = time.time()
        img = Variable(img)
        label = Variable(label).type(torch.long)
        output = conv(img)
        optimizer.zero_grad()
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        acc += (torch.max(output, dim=1)[1]==label).sum()
        count += img.size(0)
        print(count, time.time() - t)
    print(epoch_number, count, running_loss, int(acc)/count)
    epoch_loss.append(running_loss)

10 4.353150367736816
20 3.571099281311035
30 3.606004238128662
40 3.5511584281921387
50 3.4439401626586914
60 3.501784324645996
70 3.598034143447876
80 3.71821928024292
90 3.6109979152679443
100 3.507758378982544
110 3.439464569091797
120 3.3860960006713867
130 3.328735113143921
140 3.427983045578003
150 3.4311516284942627
160 3.4838194847106934
170 3.5970335006713867
180 3.527223825454712
190 3.3950650691986084
198 2.795151948928833
0 198 35.40801751613617 0.30303030303030304


In [ ]:
plt.plot(range(epoch), epoch_loss)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [6]:
torch.save(conv, save_model_name)

In [ ]:
count, acc = 0, 0.
for img, label in testloader:
    img = Variable(img)
    label = Variable(label)
    output = conv(img)
    acc += (torch.max(output, dim=1)[1] == label).sum()
    count += img.size(0)
print(count, running_loss, int(acc)/count)